In [10]:
# import library
from serpapi import GoogleSearch
from dotenv import load_dotenv
import os
import requests
import json
import pandas as pd
import requests
from PIL import Image
from io import BytesIO


load_dotenv()

SERP_API_KEY = os.getenv("SERP_API_KEY")

In [12]:
# get the search query from csv
us_state = pd.read_csv('usstate.csv')
# concat the state and document type
search_query = us_state['State_name'] + ' ' + us_state['Document_name']
search_query = search_query.to_list()

search_query = ["Arkansas Driver's License"]
print(search_query)

["Arkansas Driver's License"]


In [13]:
def getSerpAPI(query):
    params = {
      "engine": "google_images",
      "google_domain": "google.com",
      "q": query,
      "hl": "en",
      "gl": "us",
      "api_key": SERP_API_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    # save result to json file
    with open(f"{query}.json", "w") as file:
        json.dump(results, file)
    # print results
    print(results)
    return results


In [14]:
def downloadImage(results, query):
    # in case lose the result file, we can load it from the json file
    # with open(f"{query}.json", "r") as file:
    #     results = json.load(file)
    # Create directory if it doesn't exist
    if not os.path.exists(query):
        os.makedirs(query)

    for image_result in results['images_results']:
        url = ""
        if 'original' not in image_result:
            url = image_result['thumbnail']
        else:
            url = image_result['original']
        print(f"Attempting to download {url}")
        
        try:
            # Send a HTTP request to the URL of the image
            response = requests.get(url, timeout=5)  # set a timeout for the request
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}. Skipping.")
            continue

        try:
            # Open the image from the response content
            img = Image.open(BytesIO(response.content))
            
            # Convert the image to JPG format
            img_jpg = img.convert("RGB")
            
            # Save the image in JPG format
            with open(f'{query}/image_{image_result["position"]}.jpg', 'wb') as file:
                img_jpg.save(file)
                
            print("Downloaded")
        except Exception as e:
            print(f"Failed to process the image: {e}. Skipping.")


In [15]:
# Loop search query
for query in search_query:
    results = getSerpAPI(query)
    downloadImage(results, query)

{'search_metadata': {'id': '661eb4f233b236bcf5962919', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/5a7424e66c42952e/661eb4f233b236bcf5962919.json', 'created_at': '2024-04-16 17:27:14 UTC', 'processed_at': '2024-04-16 17:27:14 UTC', 'google_images_url': 'https://www.google.com/search?q=Arkansas+Driver%27s+License&oq=Arkansas+Driver%27s+License&hl=en&gl=us&tbm=isch', 'raw_html_file': 'https://serpapi.com/searches/5a7424e66c42952e/661eb4f233b236bcf5962919.html', 'total_time_taken': 2.37}, 'search_parameters': {'engine': 'google_images', 'q': "Arkansas Driver's License", 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'image_results_state': 'Results for exact spelling'}, 'suggested_searches': [{'name': 'Real id', 'link': 'https://www.google.com/search?sca_esv=1a57d827cf09faae&q=arkansas+driver%27s+license+real+id&uds=AMwkrPuuizpAfYjpZTM1FOxs7QgSf64ECVyjuJ1alz6RI6y8hl6lbqnE6V9KIr3HmzH1pJRlLSeZPVHbuvwnNXekPntq4cXtmBY

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x00000296F72F2070>